In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import psycopg2
import pandas as pd
import configparser

In [3]:
from sql_postgres import insert_table_queries, load_staging_queries

In [4]:
def load_staging_tables(engine, bucket, key, region, source_format, access_key, secret_key, ):
    cur = engine.cursor()
    for q in load_staging_queries:
        q_settings = dict(
            source_format=source_format,
            bucket=bucket,
            key=key,
            region=region,
            access_key=access_key,
            secret_key=secret_key,
        )
        q = q.format(**q_settings)
        
        cur.execute(q)
    engine.commit()

In [5]:
def insert_tables(engine):
    cur = engine.cursor()
    for q in insert_table_queries:
        cur.execute(q)
    engine.commit()

In [6]:
secrets = configparser.ConfigParser()
secrets.read('secrets.cfg')

['secrets.cfg']

In [7]:
data_cfg = configparser.ConfigParser()
data_cfg.read('data.cfg')

['data.cfg']

In [15]:
engine = psycopg2.connect(
    database=secrets["postgres"]["database"],
    user=secrets["postgres"]["user"],
    password=secrets["postgres"]["password"],
    host=secrets["postgres"]["host"],
    port=secrets["postgres"]["port"],
)

In [9]:
data_source = "sales_raw_test"

In [12]:
load_staging_tables(
    engine, 
    source_format=data_cfg[data_source]["source_format"],
    bucket=data_cfg[data_source]["bucket"],
    key=data_cfg[data_source]["key"],
    region=data_cfg[data_source]["region"],
    access_key=secrets["aws"]["access_key"],
    secret_key=secrets["aws"]["secret_key"],
)


In [13]:
insert_tables(engine)

In [16]:
cur = engine.cursor()


In [23]:
print(cur)

<cursor object at 0x7f7a932db550; closed: 0>


In [32]:
engine.close()

In [33]:
cur.close()

In [17]:
cur.execute("SELECT COUNT(*) FROM items")
results = cur.fetchall()

In [19]:
results

[(0,)]

In [28]:
for r in results:
    print(r)

(1583,)
